<a href="https://colab.research.google.com/github/federicocampo/CNN_prova/blob/Prime_modifiche/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from matplotlib import pyplot as plt
from skimage.io import imread, imshow
from skimage import img_as_float
from skimage.restoration import denoise_wavelet
from PIL import Image
import pywt
import os
import glob
import numpy as np
import logging
import time

In [ ]:
PATH = 'gdrive/MyDrive/IMAGES/Mammography_micro'

#Two and three levels denoise

In [ ]:
def twolvldenoiseddwt(myim, wavelet):
    """ This function decompose the original image with a Discrete Wavelet Transformation
        using the desired wavelet family up to the third level. It keeps all the details
        coefficient and mask the resulted approximated image in order to enhance the
        visibility of all the details.
    """
    myim = img_as_float(myim)
    myim_denoised = denoise_wavelet(myim, method='BayesShrink', mode='soft', rescale_sigma='True')

    level = 2
    # mode = 'periodization' 

    # Here I get my approximated image and the relative coefficients
    cA, (cH2, cV2, cD2), (cH1, cV1, cD1) = pywt.wavedec2(myim_denoised, wavelet, level=level)

    """ Now, I get the standard deviation for each matrix (image and coefficients).
        The std will act as a treshold so that if abs(value) < 0. --> value = 0.
                                             elif abs(value) > 0. --> value = value
    """ 
    mult_val = 1.

    ncA = np.zeros_like(cA)

    std10 = np.std(cH1)*mult_val
    std11 = np.std(cV1)*mult_val
    std12 = np.std(cD1)*mult_val

    ncH1 = pywt.threshold(cH1, std10, mode = 'hard', substitute = 0.)
    ncV1 = pywt.threshold(cV1, std11, mode = 'hard', substitute = 0.)
    ncD1 = pywt.threshold(cD1, std12, mode = 'hard', substitute = 0.)

    std20 = np.std(cH2)*mult_val
    std21 = np.std(cV2)*mult_val
    std22 = np.std(cD2)*mult_val

    ncH2 = pywt.threshold(cH2, std20, mode = 'hard', substitute = 0.)
    ncV2 = pywt.threshold(cV2, std21, mode = 'hard', substitute = 0.)
    ncD2 = pywt.threshold(cD2, std22, mode = 'hard', substitute = 0.)

    """ To let things be more readable I define new_coeff,
        this is just so that waverec2 (the function needed to reconstruct
        the image from a set of given coefficient) can do what it does.
    """

    new_coeff = ncA, (ncH2, ncV2, ncD2), (ncH1, ncV1, ncD1) 

    mynewim = pywt.waverec2(new_coeff, wavelet)
    mynewim = pywt.threshold(mynewim, 0., mode = 'greater', substitute = 0.)

    return mynewim

In [ ]:
def threelvldenoiseddwt(myim, wavelet):
    """ This function decompose the original image with a Discrete Wavelet Transformation
        using the desired wavelet family up to the third level. It keeps all the details
        coefficient and mask the resulted approximated image in order to enhance the
        visibility of all the details.
    """
    myim = img_as_float(myim)
    myim_denoised = denoise_wavelet(myim, method='BayesShrink', mode='soft', rescale_sigma='True')

    level = 3
    # mode = 'periodization' 

    # Here I get my approximated image and the relative coefficients
    cA, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = pywt.wavedec2(myim_denoised, wavelet, level=level)

    """ Now, I get the standard deviation for each matrix (image and coefficients).
        The std will act as a treshold so that if abs(value) < 0. --> value = 0.
                                             elif abs(value) > 0. --> value = value
    """ 
    mult_val = 1.

    ncA = np.zeros_like(cA)

    std10 = np.std(cH1)*mult_val
    std11 = np.std(cV1)*mult_val
    std12 = np.std(cD1)*mult_val

    ncH1 = pywt.threshold(cH1, std10, mode = 'hard', substitute = 0.)
    ncV1 = pywt.threshold(cV1, std11, mode = 'hard', substitute = 0.)
    ncD1 = pywt.threshold(cD1, std12, mode = 'hard', substitute = 0.)

    std20 = np.std(cH2)*mult_val
    std21 = np.std(cV2)*mult_val
    std22 = np.std(cD2)*mult_val

    ncH2 = pywt.threshold(cH2, std20, mode = 'hard', substitute = 0.)
    ncV2 = pywt.threshold(cV2, std21, mode = 'hard', substitute = 0.)
    ncD2 = pywt.threshold(cD2, std22, mode = 'hard', substitute = 0.)
    
    std30 = np.std(cH3)*mult_val
    std31 = np.std(cV3)*mult_val
    std32 = np.std(cD3)*mult_val

    ncH3 = pywt.threshold(cH3, std30, mode = 'hard', substitute = 0.)
    ncV3 = pywt.threshold(cV3, std31, mode = 'hard', substitute = 0.)
    ncD3 = pywt.threshold(cD3, std32, mode = 'hard', substitute = 0.)

    """ To let things be more readable I define new_coeff,
        this is just so that waverec2 (the function needed to reconstruct
        the image from a set of given coefficient) can do what it does.
    """

    new_coeff = ncA, (ncH3, ncV3, ncD3), (ncH2, ncV2, ncD2), (ncH1, ncV1, ncD1) 

    mynewim = pywt.waverec2(new_coeff, wavelet)
    mynewim = pywt.threshold(mynewim, 0., mode = 'greater', substitute = 0.)

    return mynewim

#Reading images

In [ ]:
import multiprocessing as mp

def read_img(image_path):
  '''Takes as input the path to the image folder and 
  returns the numpy array of images and label found in that folder'''

  #Creating a list of all image names found in image_path
  fnames = glob.glob(os.path.join(image_path, '*.pgm'))

  #Defining 4 sub-processes and apply imread to all the images found previously
  #(imread reads images in pgm format)
  pool = mp.Pool(processes=4)
  results = pool.map_async(imread, fnames)

  #Get the list of images and convert to numpy array
  x = results.get()
  x_np = np.array(x, dtype='float32')[..., np.newaxis]/255

  logger.info('Num images found in %s: %d',image_path, len(x_np))

  #Create a list of corrisponding labels and conver it to numpy array
  label = os.path.basename(image_path)
  y = [int(label)] * len(x_np)
  y_np = np.array(y)

  
  return x_np, y_np

In [ ]:
'''Read images using read_img function'''

start_time = time.time()

#Define the path to the sub-folder of Train images folder containing "normal" breast mammograms
image_path = os.path.join(PATH, 'Train/0')
#Create the test images and labels array with read_img function
x0_train, y0_train = read_img(image_path)


#Define the path to the sub-folder of Train images folder containing breast mammograms with microcalcifications
image_path = os.path.join(PATH, 'Train/1')
#Create the test images and labels array with read_img function
x1_train, y1_train = read_img(image_path)

#Create an array with both normal and sick images and labels
x_train = np.concatenate((x0_train, x1_train), axis = 0)
y_train = np.concatenate((y0_train, y1_train))

#Processing images
proc_x_train = []
for image in enumerate(x_train):
  processedimage = threelvldenoiseddwt(image, 'db5')
  proc_x_train.append(processedimage)

proc_x_train = np.array(processed_x_train, dtype='float32')[..., np.newaxis]/255



#Doing the same of previous lines, on Test folder
image_path = os.path.join(PATH, 'Test/0')
x0_test, y0_test = read_img(image_path)

image_path = os.path.join(PATH, 'Test/1')
x1_test, y1_test = read_img(image_path)

x_test = np.concatenate((x0_test, x1_test), axis = 0)
y_test = np.concatenate((y0_test, y1_test))

proc_x_test = []
for image in enumerate(x_test):
  processedimage = threelvldenoiseddwt(image, 'db5')
  proc_x_test.append(processedimage)

proc_x_test = np.array(processed_x_test, dtype='float32')[..., np.newaxis]/255


#Print the total number of images found.
print(f'There are {len(x_train)} train images and {len(x_test)} test images')

elapsed_time = time.time() - start_time
logger.debug('Done in %.2f s', elapsed_time)

print(f'\nx_test_processed shape = {proc_x_test.shape} <-- (number of images, width, height)\n')

# Defining the model

In [ ]:
from keras.layers import Dense, Flatten, BatchNormalization
from keras.models import Sequential

In [ ]:
model = Sequential([
    Flatten(), 
    Dense(102,  activation = 'relu'), 
    BatchNormalization(),
    Dense(32, activation = 'relu'), 
    BatchNormalization(),

    Dense(1, activation = 'sigmoid')
])

Compile and train

In [ ]:
from keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate = 0.0005),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    x_train, y_train,
    validation_split = 0.25, 
    epochs=50,
)

In [ ]:
'''Visualize loss, val_loss, accuracy and val_accuracy obtanined during the train''' 
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss and val_loss')

plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy and val_accuracy')